In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import the CRUD module
from my_project import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Use the established credentials to access
username = "aacuser"
password = "letmelearn1"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
print(len(df.to_dict(orient='records')))
print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Branded logo data
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Place the image and my name at the top, above the data tables and visualizations
app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Img(src="data:image/png;base64,{}".format(encoded_image.decode()), style={"height": "100px"}),
    html.Center(html.B(html.H1('CS-340 Dashboard - Alexander Flood'))),
    html.Hr(),
    # Declare the filter selection
    html.Div([
        html.H4("Select Rescue Type:"),
        dcc.RadioItems(
            id="filter-type", # Declare an ID to read the radio state
            options=[ # An option for each rescue type
                {"label": "Water Rescue", "value": "Water Rescue"},
                {"label": "Mountain or Wilderness Rescue", "value": "Mountain or Wilderness Rescue"},
                {"label": "Disaster or Individual Tracking", "value": "Disaster or Individual Tracking"},
                {"label": "Reset", "value": "Reset"}
            ],
            value="Reset", # Default state
            labelStyle={"display": "block"}
        )
    ]),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        style_table={'overflowX': 'auto'},
        page_size=10, # 10 results per page
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False, # No deletion
        selected_columns=[],
        selected_rows=[0], # Start with the first row selected
        page_action="native",
        page_current=0 # Start on the first page
    ),
    html.Br(),
    html.Hr(),
    # This sets up the dashboard so that the chart and the geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
# Display the breeds of animal based on quantity represented in
# the data table
    if filter_type == 'Water Rescue':
        # If Water Rescue, display water rescue breeds
        query = {
            "$and": [
                {"breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}},
                {"sex_upon_outcome": "Intact Female"},
                {"age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
            ]
        }
    elif filter_type == 'Mountain or Wilderness Rescue':
        # If Mountain or Wilderness Rescue, display m & w breeds
        query = {
            "$and": [
                {"breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}},
                {"sex_upon_outcome": "Intact Male"},
                {"age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
            ]
        }
    elif filter_type == 'Disaster or Individual Tracking':
        # If Disaster or Individual Tracking, display disaster and individual tracking breeds
        query = {
            "$and": [
                {"breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}},
                {"sex_upon_outcome": "Intact Male"},
                {"age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}}
            ]
        }
    else:
        query = {} # No applied filter

    # C(R)UD - run the read query
    data = db.read(query)
    
    # Place it in a dataframe
    df = pd.DataFrame.from_records(data)
    
    # Remove MongoDB _id field
    df.drop(columns=['_id'], inplace=True, errors='ignore')

    # Return the results
    return df.to_dict('records')
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    # If no viewData, inform the user there is no data to display
    if viewData is None or len(viewData) == 0:
        return html.Div("No data to display.")
    
    # Load the data into a dataframe
    dff = pd.DataFrame.from_dict(viewData)
    
    # Get the breed counts
    breed_counts = dff["breed"].value_counts().reset_index()
    breed_counts.columns = ["breed", "count"] # Set the columns
    
    # Create the figure
    fig = px.pie(breed_counts, names="breed", values="count", title="Rescue Animal Breed")
    
    # Update
    fig.update_traces(textposition="inside", textinfo="percent+label")

    return [
        dcc.Graph(figure=fig)
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    # If not viewData or index, return
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


50000
Index(['rec_num', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed',
       'color', 'date_of_birth', 'datetime', 'monthyear', 'name',
       'outcome_subtype', 'outcome_type', 'sex_upon_outcome', 'location_lat',
       'location_long', 'age_upon_outcome_in_weeks'],
      dtype='object')
Dash app running on http://127.0.0.1:27979/
